In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import country_converter as coco

In [2]:
df = pd.read_csv("../countries.csv")
agro = pd.read_csv("../Agroalimentaire/global-food-prices.csv")

/tmp/ipykernel_16795/1255250984.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  agro = pd.read_csv("../Agroalimentaire/global-food-prices.csv")


In [3]:
def perf_coco(df2):
    """
    Fonction perf_coco -
    Paramètre : 
    - df2 : colonne de nom de pays à valeur unique

    Sortie : Comparaison entre aucun changement, coco_convert en name short et coco_convert en ISO2
    """
    print("===== Performance sans changement : ======")
    set_pays = set(df["Name"])
    set_df2 = set(df2)
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")

    print("===== Performance short name pays : ======")
    set_df2 = set(coco.convert(names=df2,to='name_short'))
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")

    print("===== Performance code pays : ======")
    set_pays = set(df["Code"])
    set_df2 = set(coco.convert(names=df2,to='ISO2'))
    valeurs_communes = set_pays.intersection(set_df2)
    print(f"Valeur communes : {len(valeurs_communes)}/{len(df2)}")


In [4]:
pays_agro = pd.unique(agro["adm0_name"])
perf_coco(pays_agro)

Bassas da India not found in regex
Bassas da India not found in regex


===== Performance sans changement : ======
Valeur communes : 71/78
===== Performance short name pays : ======
Valeur communes : 64/78
===== Performance code pays : ======
Valeur communes : 77/78


In [5]:
eco = pd.read_csv("./global_gdp.csv")
lst = pd.unique(eco["Entity"])
perf_coco(lst)

===== Performance sans changement : ======
Valeur communes : 175/208
===== Performance short name pays : ======


East Asia and Pacific (WB) not found in regex
Europe and Central Asia (WB) not found in regex
European Union (27) not found in regex
High-income countries not found in regex
Latin America and Caribbean (WB) not found in regex
Low-income countries not found in regex
Lower-middle-income countries not found in regex
Middle East and North Africa (WB) not found in regex
Middle-income countries not found in regex
North America (WB) not found in regex
South Asia (WB) not found in regex
Sub-Saharan Africa (WB) not found in regex
Upper-middle-income countries not found in regex
World not found in regex
East Asia and Pacific (WB) not found in regex
Europe and Central Asia (WB) not found in regex
European Union (27) not found in regex
High-income countries not found in regex
Latin America and Caribbean (WB) not found in regex
Low-income countries not found in regex
Lower-middle-income countries not found in regex
Middle East and North Africa (WB) not found in regex
Middle-income countries not fou

Valeur communes : 168/208
===== Performance code pays : ======


Sub-Saharan Africa (WB) not found in regex
Upper-middle-income countries not found in regex
World not found in regex


Valeur communes : 192/208


In [6]:
wage = pd.read_csv("./minimum_wage.csv")
perf_coco(wage["country"])

===== Performance sans changement : ======
Valeur communes : 138/154
===== Performance short name pays : ======
Valeur communes : 133/154
===== Performance code pays : ======
Valeur communes : 154/154


In [7]:
## On voit qu'il y le plus de ressemblance grâce à ISO2, on utilisera
## cette standardisation pour la suite.

def standardisation(col):
    """
    Fonction qui prend en paramètre une colonne nom et la transforme en norme ISO2 
    """
    cc = coco.CountryConverter()
    col = cc.convert(col,to = 'ISO3')
    return col

In [8]:
wage["country"] = standardisation(wage["country"])

In [9]:
result = pd.merge(eco,wage,left_on="Code", right_on="country", how="left")
result

,Entity,Code,Year,"GDP per capita, PPP (constant 2017 international $)",country,minimumWageByCountry_annualMinimalWageEarnings,minimumWageByCountry_hourlyMinimalWageEarnings
0,Afghanistan,AFG,2002,1280.4631,AFG,858.0,0.41
1,Afghanistan,AFG,2003,1292.3335,AFG,858.0,0.41
2,Afghanistan,AFG,2004,1260.0605,AFG,858.0,0.41
3,Afghanistan,AFG,2005,1352.3207,AFG,858.0,0.41
4,Afghanistan,AFG,2006,1366.9932,AFG,858.0,0.41
...,...,...,...,...,...,...,...
6557,Zimbabwe,ZWE,2018,2399.6216,NaN,NaN,NaN
6558,Zimbabwe,ZWE,2019,2203.3967,NaN,NaN,NaN
6559,Zimbabwe,ZWE,2020,1990.3195,NaN,NaN,NaN
6560,Zimbabwe,ZWE,2021,2115.1445,NaN,NaN,NaN


In [10]:
## ================= Unemployment table =================
unemployment = pd.read_csv("./Unemployment.csv")
#On supprime les lignes vide de Country code qui sont 1) des données non pratiquables 2) la source des données
unemployment = unemployment[unemployment['Country Code'].notna()]
#Et on supprime les colonnes Series et Series Code qui sont inutiles pour l'avenir
unemployment = unemployment.drop(['Series','Series Code'], axis=1)


In [11]:
#On fait en sorte que les années soient par colonne pour standardiser les df.
UN_rate = unemployment.melt(id_vars=["Country","Country Code"],
                  var_name="Year",
                  value_name="UnemploymentRate")

year = UN_rate["Year"]

In [12]:
#On standardise la colonne année.
UN_rate["Year"] = UN_rate["Year"].str.slice(0,4)

In [13]:
#On change le type de Year pour correspondre à la colonne dans result
UN_rate["Year"] = UN_rate["Year"].astype('int')

In [14]:
result = result.merge(UN_rate,how="left", left_on=["Code","Year"], right_on=["Country Code","Year"])

In [15]:
result[result["Entity"] == "France"]

,Entity,Code,Year,"GDP per capita, PPP (constant 2017 international $)",country,minimumWageByCountry_annualMinimalWageEarnings,minimumWageByCountry_hourlyMinimalWageEarnings,Country,Country Code,UnemploymentRate
2055,France,FRA,1990,33843.020,FRA,24259.0,13.33,NaN,NaN,NaN
2056,France,FRA,1991,33898.234,FRA,24259.0,13.33,NaN,NaN,NaN
2057,France,FRA,1992,34269.285,FRA,24259.0,13.33,NaN,NaN,NaN
2058,France,FRA,1993,33906.223,FRA,24259.0,13.33,NaN,NaN,NaN
2059,France,FRA,1994,34576.344,FRA,24259.0,13.33,NaN,NaN,NaN
2060,France,FRA,1995,35176.620,FRA,24259.0,13.33,NaN,NaN,NaN
2061,France,FRA,1996,35546.582,FRA,24259.0,13.33,NaN,NaN,NaN
2062,France,FRA,1997,36247.605,FRA,24259.0,13.33,NaN,NaN,NaN
2063,France,FRA,1998,37409.395,FRA,24259.0,13.33,NaN,NaN,NaN
2064,France,FRA,1999,38490.040,FRA,24259.0,13.33,NaN,NaN,NaN


In [16]:
result = result.drop(columns=["Country","Country Code","country"])

In [17]:
result.to_csv("result.csv",index=False)

In [18]:
#Traiter la table cooking / arrivaltourists
df_arrival = pd.read_csv("./arrivaltourists.csv")
#On supprime les lignes vide de Country code qui sont 1) des données non pratiquables 2) la source des données
df_arrival = df_arrival[df_arrival['Country Code'].notna()]
#Et on supprime les colonnes Series et Series Code qui sont inutiles pour l'avenir
df_arrival = df_arrival.drop(['Series Name','Series Code'], axis=1)

In [19]:
df_arrival

,Country Name,Country Code,2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Afghanistan,AFG,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,Albania,ALB,470000,557000,645000,748000,937000,1127000,1420000,1856000,...,3256000,3673000,4131000,4736000,5118000,5927000,6406000,2658000,..,..
2,Algeria,DZA,988000,1166000,1234000,1443000,1638000,1743000,1772000,1912000,...,2733000,2301000,1710000,2039000,2451000,2657000,2371000,591000,..,..
3,American Samoa,ASM,..,..,..,24500,25299.9992370605,24100.0003814697,24299.9992370605,23500,...,20799.9992370605,21600.0003814697,20299.9992370605,20100.0003814697,20000,20200.0007629395,19200.0007629395,899.999976158142,..,..
4,Andorra,AND,11507000,11601000,11668000,11049000,10737000,10743000,10194000,9112000,...,7676000,7797000,7850000,8025000,8152000,8328000,8235000,5207000,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,SSF,22162330.2629762,22926433.4292146,24699148.8251034,26367736.5442803,29762488.2419261,34665318.3696877,35737670.056065,36704327.0214213,...,44728989.8968498,46510985.5551164,48979055.5101286,51223036.3828596,52899581.2058794,56673405.0653589,56014499.1539477,..,..,..
262,Sub-Saharan Africa (excluding high income),SSA,22023890.3822426,22796021.3552279,24570128.1884486,26229482.2972481,29607860.52355,34490086.8944156,35560286.3026162,36521793.401362,...,44484730.5513001,46265055.5840422,48672074.1038622,50877194.3830237,52496629.8574719,56247033.3736039,55555231.2954125,..,..,..
263,Sub-Saharan Africa (IDA & IBRD countries),TSS,22162330.2629762,22926433.4292146,24699148.8251034,26367736.5442803,29762488.2419261,34665318.3696877,35737670.056065,36704327.0214213,...,44728989.8968498,46510985.5551164,48979055.5101286,51223036.3828596,52899581.2058794,56673405.0653589,56014499.1539477,..,..,..
264,Upper middle income,UMC,323043770.078842,308581283.043352,351774849.750366,377443418.735845,385922932.173933,408189869.124966,417498838.987031,406075035.229862,...,455797691.588978,465397898.821266,484482005.838454,498471214.239233,534415314.856608,556608632.209134,573310641.28494,..,..,..


In [20]:
df_arrival = df_arrival.melt(id_vars=["Country Name","Country Code"],
                  var_name="Year",
                  value_name="Arrival")

df_arrival

,Country Name,Country Code,Year,Arrival
0,Afghanistan,AFG,2002 [YR2002],..
1,Albania,ALB,2002 [YR2002],470000
2,Algeria,DZA,2002 [YR2002],988000
3,American Samoa,ASM,2002 [YR2002],..
4,Andorra,AND,2002 [YR2002],11507000
...,...,...,...,...
5581,Sub-Saharan Africa,SSF,2022 [YR2022],..
5582,Sub-Saharan Africa (excluding high income),SSA,2022 [YR2022],..
5583,Sub-Saharan Africa (IDA & IBRD countries),TSS,2022 [YR2022],..
5584,Upper middle income,UMC,2022 [YR2022],..


In [21]:
df_arrival["Year"] = df_arrival["Year"].str.slice(0,4)
#On change le type de Year pour correspondre à la colonne dans result
df_arrival["Year"] = df_arrival["Year"].astype('int')

In [22]:
result = result.merge(df_arrival,how="left", left_on=["Code","Year"], right_on=["Country Code","Year"])

In [23]:
result = result.drop(columns=["Country Name","Country Code"])
result.to_csv("result.csv",index=False)